# **TASK-3 by XGBoost Model and AdaBoost Model**
This file contains task-3 operations for problem statement-2(referenced from readme) performed by using XGBoost model and AdaBoost model as mentioned in the question.


# DATA PREPROCESSING AND DATA TRANSFORMATION

## Importing all the needed libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score ,classification_report ,confusion_matrix, precision_score,recall_score, f1_score
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

## Creating a dataframe for smoking and drinking dataset

In [3]:
df  = pd.read_csv('/content/drive/MyDrive/Datasets/smoking_driking_dataset_Ver01.csv')
df

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991341,Male,45,175,80,92.1,1.5,1.5,1.0,1.0,114.0,...,125.0,132.0,15.0,1.0,1.0,26.0,36.0,27.0,1.0,N
991342,Male,35,170,75,86.0,1.0,1.5,1.0,1.0,119.0,...,84.0,45.0,15.8,1.0,1.1,14.0,17.0,15.0,1.0,N
991343,Female,40,155,50,68.0,1.0,0.7,1.0,1.0,110.0,...,77.0,157.0,14.3,1.0,0.8,30.0,27.0,17.0,3.0,Y
991344,Male,25,175,60,72.0,1.5,1.0,1.0,1.0,119.0,...,73.0,53.0,14.5,1.0,0.8,21.0,14.0,17.0,1.0,N


## Checking for null values

In [4]:
df.isnull().sum()

sex                 0
age                 0
height              0
weight              0
waistline           0
sight_left          0
sight_right         0
hear_left           0
hear_right          0
SBP                 0
DBP                 0
BLDS                0
tot_chole           0
HDL_chole           0
LDL_chole           0
triglyceride        0
hemoglobin          0
urine_protein       0
serum_creatinine    0
SGOT_AST            0
SGOT_ALT            0
gamma_GTP           0
SMK_stat_type_cd    0
DRK_YN              0
dtype: int64

As observed above there are no null values present in any of the columns in the dataset.



## Checking for duplicate rows and removing them

In [5]:
df.duplicated().value_counts()

False    991320
True         26
dtype: int64

In [6]:
df.drop_duplicates()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991341,Male,45,175,80,92.1,1.5,1.5,1.0,1.0,114.0,...,125.0,132.0,15.0,1.0,1.0,26.0,36.0,27.0,1.0,N
991342,Male,35,170,75,86.0,1.0,1.5,1.0,1.0,119.0,...,84.0,45.0,15.8,1.0,1.1,14.0,17.0,15.0,1.0,N
991343,Female,40,155,50,68.0,1.0,0.7,1.0,1.0,110.0,...,77.0,157.0,14.3,1.0,0.8,30.0,27.0,17.0,3.0,Y
991344,Male,25,175,60,72.0,1.5,1.0,1.0,1.0,119.0,...,73.0,53.0,14.5,1.0,0.8,21.0,14.0,17.0,1.0,N


Also as observed in the EDA file there are **no unknown values** in this dataset and hence imputation is not required.

##Encoding


### Label Encoding
Here columns 'Sex',  'SMK_stat_type_cd',  'DRK_YN' have values with string data-type and will be label encoded.

In [7]:
le = LabelEncoder()

df["sex"] = le.fit_transform(df["sex"])
df["SMK_stat_type_cd"] = le.fit_transform(df["SMK_stat_type_cd"])
df["DRK_YN"] = le.fit_transform(df["DRK_YN"])
df['SMK_stat_type_cd'] = df['SMK_stat_type_cd'].astype('int')

## Correlation Matrix

In [8]:
df.corr()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
sex,1.000000,-0.083225,0.722774,0.581707,0.344725,0.099016,0.101522,0.002926,-0.000751,0.176245,...,-0.005210,0.216283,0.669417,0.020184,0.245956,0.092943,0.196751,0.257229,0.591004,0.368726
age,-0.083225,1.000000,-0.398501,-0.195337,0.127170,-0.172096,-0.167684,0.203021,0.198693,0.265530,...,0.029497,0.043549,-0.173081,0.041497,0.022819,0.059408,-0.020506,0.017391,-0.126031,-0.284587
height,0.722774,-0.398501,1.000000,0.668823,0.263945,0.139141,0.138529,-0.081159,-0.082456,0.035030,...,-0.015450,0.137611,0.531898,0.002298,0.171256,0.039203,0.144842,0.162340,0.467317,0.374566
weight,0.581707,-0.195337,0.668823,1.000000,0.637173,0.088901,0.088707,-0.044498,-0.045709,0.250770,...,0.067859,0.283774,0.499491,0.029281,0.153885,0.099652,0.276440,0.221881,0.366499,0.264282
waistline,0.344725,0.127170,0.263945,0.637173,1.000000,0.004511,0.006158,0.033224,0.031012,0.272323,...,0.063416,0.249436,0.291730,0.043132,0.098421,0.096858,0.212267,0.186988,0.205485,0.092993
sight_left,0.099016,-0.172096,0.139141,0.088901,0.004511,1.000000,0.307985,-0.039428,-0.040998,-0.035617,...,0.003013,0.010599,0.085896,-0.010253,0.020309,-0.005427,0.018815,0.013566,0.064194,0.077379
sight_right,0.101522,-0.167684,0.138529,0.088707,0.006158,0.307985,1.000000,-0.040764,-0.038895,-0.033994,...,0.002155,0.012265,0.086847,-0.010897,0.021447,-0.004600,0.018920,0.016491,0.065051,0.074806
hear_left,0.002926,0.203021,-0.081159,-0.044498,0.033224,-0.039428,-0.040764,1.000000,0.536890,0.056065,...,-0.014621,0.002095,-0.036058,0.019754,0.015254,0.011283,-0.007907,0.002991,-0.021245,-0.058337
hear_right,-0.000751,0.198693,-0.082456,-0.045709,0.031012,-0.040998,-0.038895,0.536890,1.000000,0.057436,...,-0.013438,0.002155,-0.036703,0.018264,0.012959,0.010763,-0.008687,0.003548,-0.022330,-0.058230
SBP,0.176245,0.265530,0.035030,0.250770,0.272323,-0.035617,-0.033994,0.056065,0.057436,1.000000,...,0.033619,0.186003,0.166530,0.063518,0.062605,0.081478,0.117613,0.161434,0.084918,0.033149


## Splitting Dataset

In [9]:
a=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23]
x=df.iloc[:,a]
x = pd.get_dummies(x, columns=['sex'])
y = df.iloc[:,22]


In [10]:
x

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,DRK_YN,sex_0,sex_1
0,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,80.0,...,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1,0,1
1,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,82.0,...,121.0,15.8,1.0,0.9,20.0,36.0,27.0,0,0,1
2,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,70.0,...,104.0,15.8,1.0,0.9,47.0,32.0,68.0,0,0,1
3,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,87.0,...,106.0,17.6,1.0,1.1,29.0,34.0,18.0,0,0,1
4,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,82.0,...,104.0,13.8,1.0,0.8,19.0,12.0,25.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991341,45,175,80,92.1,1.5,1.5,1.0,1.0,114.0,80.0,...,132.0,15.0,1.0,1.0,26.0,36.0,27.0,0,0,1
991342,35,170,75,86.0,1.0,1.5,1.0,1.0,119.0,83.0,...,45.0,15.8,1.0,1.1,14.0,17.0,15.0,0,0,1
991343,40,155,50,68.0,1.0,0.7,1.0,1.0,110.0,70.0,...,157.0,14.3,1.0,0.8,30.0,27.0,17.0,1,1,0
991344,25,175,60,72.0,1.5,1.0,1.0,1.0,119.0,74.0,...,53.0,14.5,1.0,0.8,21.0,14.0,17.0,0,0,1


In [11]:
y

0         0
1         2
2         0
3         0
4         0
         ..
991341    0
991342    0
991343    2
991344    0
991345    2
Name: SMK_stat_type_cd, Length: 991346, dtype: int64

## Train-Test Split

Splitting our data into train and test datasets.
Here testing dataset contains 20% of all the data and training dataset contains the remaining 80%.

In [12]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=0)


## Scaling data by use of Standardization



In [13]:
st_x= StandardScaler()
x_train= st_x.fit_transform(x_train)
x_test= st_x.transform(x_test)

# XGBOOST MODEL BUILDING

## Training our Model

In [ ]:


model = xgb.XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

## MODEL EVALUATION

### Accuracy

In [ ]:
accuracy_score(y_test,y_pred)

### Precision

In [48]:
precision_score(y_test,y_pred, average="weighted")

0.7026967798881469

### Recall

In [49]:
recall_score(y_test,y_pred, average="weighted")


0.7081000655671559

### F1 Score

In [50]:
f1_score(y_test,y_pred, average="weighted")


0.7042803932047681

### Classification Report

In [51]:
report = classification_report(y_test, y_pred , output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.832872,0.846552,0.839656,120425.0000
1,0.460749,0.375135,0.413558,35078.0000
2,0.534593,0.591344,0.561538,42767.0000
accuracy,0.708100,0.708100,0.708100,0.7081
macro avg,0.609405,0.604344,0.604917,198270.0000
weighted avg,0.702697,0.708100,0.704280,198270.0000


## Hyper-parameter Tuning

In [ ]:
param_grid = {
    'max_depth': [5,6,7,8,9],
    'learning_rate': [0.11,0.12,0.13,0.14],
    'subsample': [ 0.7,0.8,0.9,1]
}


grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

grid_search.fit(x_train, y_train)

print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

## Model Evaluation after Hyper-parameter Tuning

### Accuracy

In [ ]:
accuracy_score(y_test,y_pred)

### Precision

In [ ]:
precision_score(y_test,y_pred, average="weighted")

### Recall

In [ ]:
recall_score(y_test,y_pred, average="weighted")


### F1 Score

In [ ]:
f1_score(y_test,y_pred, average="weighted")


### Classification Report

In [ ]:
report = classification_report(y_test, y_pred , output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

# ADABOOST MODEL BUILDING

In [15]:
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1, random_state=0)
model = abc.fit(x_train, y_train)
y_pred = model.predict(x_test)


## Model Evaluation

### Accuracy

In [16]:
accuracy_score(y_test,y_pred)

0.6981590760074645

### Precision

In [17]:
precision_score(y_test,y_pred, average="weighted")

0.6892140610161926

### Recall

In [18]:
recall_score(y_test,y_pred, average="weighted")


0.6981590760074645

### F1 Score

In [19]:
f1_score(y_test,y_pred, average="weighted")


0.6901186361349001

### Classification Report

In [20]:
report = classification_report(y_test, y_pred , output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.822431,0.847789,0.834918,120425.000000
1,0.454561,0.304863,0.364958,35078.000000
2,0.506560,0.599411,0.549088,42767.000000
accuracy,0.698159,0.698159,0.698159,0.698159
macro avg,0.594518,0.584021,0.582988,198270.000000
weighted avg,0.689214,0.698159,0.690119,198270.000000
